In [38]:
from osgeo import gdal
from sklearn import svm
import numpy
import time
from osgeo import gdal
from sklearn.externals import joblib
from numpy.core.umath import add, subtract
import gc
from multiprocessing import Pool
import os

class MDimage(object):
    def __init__(self, filepath):
        self.filepath = filepath
        self.ds = gdal.Open(filepath, gdal.GA_ReadOnly)
        self.iminfo = dict()
        self.iminfo['bandnum'] = self.ds.RasterCount
        self.iminfo['cols'] = self.ds.RasterXSize
        self.iminfo['rows'] = self.ds.RasterYSize
        self.iminfo['originX'] = self.ds.GetGeoTransform()[0]
        self.iminfo['originY'] = self.ds.GetGeoTransform()[3]
        self.iminfo['pixelWidth'] = self.ds.GetGeoTransform()[1]
        self.iminfo['pixelHeight'] = self.ds.GetGeoTransform()[5]

class Index(MDimage):
    def __init__(self, filepath):
        self.filepath = filepath
        super(Index, self).__init__(filepath)

    def index2Array(self):
        cols = self.iminfo['cols']
        rows = self.iminfo['rows']
        band = self.ds.GetRasterBand(1)
        array = band.ReadAsArray(0, 0, cols, rows)
        array =array/10000
        array = array.reshape(cols * rows)
        return array

    def WriteArrayAsImage(self, out_fname, outArray):
        cols = self.iminfo['cols']
        rows = self.iminfo['rows']
        driver = self.ds.GetDriver()
        outArray= outArray.reshape([rows,cols])
        outDS = driver.Create(out_fname, cols, rows, 1, gdal.GDT_Int16)
        outDS.SetGeoTransform(self.ds.GetGeoTransform())
        outDS.SetProjection(self.ds.GetProjection())
        outBand = outDS.GetRasterBand(1)
        outBand.WriteArray(outArray)
        outDS = None
        del outDS, outBand


In [8]:

fill=-999

#change file paths accordingly 


path1="/home/faizan/Pakistan/EVI/"
path2="/home/faizan/Pakistan/LSWI/"
np_dir="/home/faizan/Desktop/numpy_dir/"
for year in range(2002,2015):
    for i in range(1, 363):    

        #assigning names test data
        evi=path1+str(year)+"/clip.%d%03d"  % (year, i) + ".tif"
        lswi=path2+str(year)+"/clip.%d%03d"  % (year, i) + ".tif"
        Oflood =path1+str(year)+"/flood.%d%03d"  % (year, i) + ".tif"     

    #Test data
        if os.path.exists(lswi) is True: 
            limage = Index(lswi)
            LSWI = (limage.index2Array())
                        
            limage = Index(evi)
            cols = limage.iminfo['cols']
            rows = limage.iminfo['rows']
            bandnum= limage.iminfo['bandnum']
            EVI = (limage.index2Array())

            Data = numpy.vstack([EVI, LSWI])
            del EVI, LSWI

# change the file path 
            clf = joblib.load('/home/faizan/Desktop/model/modis_trained.pkl')

            def argwrapper(args):
                '''
                ラッパー関数
                '''
                return args[0](*args[1:])

            def myfunc(x):
                '''
                並列に計算したい関数
                '''
                return clf.predict([Data[0][x], Data[1][x], subtract(Data[0][x], Data[1][x])])

        #Process 
            start_time = time.time()
            if __name__ == '__main__':
                p = Pool(6)
                for a in xrange(0, 10):
                    func_args = []
                    for i in xrange((len(Data[0])*a)/10, (len(Data[0])*(a+1))/10):
                        func_args.append((myfunc, i))
                    results = numpy.array(p.map(argwrapper, func_args), dtype=numpy.int16)
                    del func_args
                    numpy.save(np_dir + "svm_result%02d" % a, results)
                    del results
                    print("--- %s seconds ---" % (time.time() - start_time))
                del Data
                p.close()
        # Load all array and merge
                # load first array
                ARRAY = numpy.load(np_dir + "svm_result00.npy")

                for a in range(1, 10):
                    dummy = numpy.load(np_dir + "svm_result%02d.npy" % (a))
                    ARRAY = numpy.vstack([ARRAY, dummy])
        # Write image 
                limage.WriteArrayAsImage(Oflood, ARRAY)
                del ARRAY, dummy
                gc.collect()

--- 81.680768013 seconds ---
--- 163.196515799 seconds ---
--- 241.19452095 seconds ---
--- 317.361662865 seconds ---
--- 393.169782877 seconds ---
--- 466.459987879 seconds ---
--- 545.568372965 seconds ---
--- 625.732739925 seconds ---
--- 708.38287282 seconds ---
--- 795.082197905 seconds ---


In [39]:
limage = Index("/home/faizan/Pakistan/EVI/2002/Clip.2002161.tif")
cols = limage.iminfo['cols']
rows = limage.iminfo['rows']
bandnum= limage.iminfo['bandnum']
EVI = (limage.index2Array())

In [40]:
EVI[5959709]

0.14849999999999999

In [34]:
limage = Index("/home/faizan/Pakistan/LSWI/2002/Clip.2002161.tif")
LSWI = (limage.index2Array())

In [35]:
LSWI[5959709]

-0.015113768